# FPUS23 — YOLO Noise Comparisons (Baseline & Balanced) for n/s/m

Creates denoised train sets (median blur) and trains YOLO11 nano/small/medium on:
- Baseline: original vs denoised
- Optimized: balanced vs denoised balanced
Val/test remain original for fair A/B.

In [ ]:
from google.colab import drive; drive.mount('/content/drive', force_remount=True)
DRIVE_DIR = '/content/drive/MyDrive/FPUS23_runs'
import pathlib as p; p.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
print('Saving to:', DRIVE_DIR)


In [ ]:
# GPU batch heuristic
import torch, os
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
print('GPU:', gpu)
if 'T4' in gpu: os.environ['FPUS23_BATCH']='16'
elif 'A100' in gpu: os.environ['FPUS23_BATCH']='24'
else: os.environ['FPUS23_BATCH']='12'
print('Recommended batch:', os.environ['FPUS23_BATCH'])


## Build denoised train sets via script (baseline & balanced)

In [ ]:
# Baseline denoised (writes /content/fpus23_project/dataset/fpus23_yolo_despeckled/data.yaml)
!python '/content/fpus23/scripts/preprocess_fpus23_despeckle.py' \
  --input /content/fpus23_project/dataset/fpus23_yolo/images/train \
  --output /content/fpus23_project/dataset/fpus23_yolo_despeckled/images/train \
  --kernel 5 \
  --data-yaml /content/fpus23_project/dataset/fpus23_yolo/data.yaml
# Balanced denoised (writes /content/fpus23_project/dataset/fpus23_yolo_balanced_despeckled/data.yaml)
!python '/content/fpus23/scripts/preprocess_fpus23_despeckle.py' \
  --input /content/fpus23_project/dataset/fpus23_yolo_balanced/images/train \
  --output /content/fpus23_project/dataset/fpus23_yolo_balanced_despeckled/images/train \
  --kernel 5 \
  --data-yaml /content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml


## Baseline (original vs denoised) — n/s/m

In [ ]:
BASE_YAML='/content/fpus23_project/dataset/fpus23_yolo/data.yaml'
BASE_DESP_YAML='/content/fpus23_project/dataset/fpus23_yolo_despeckled/data.yaml'
# nano
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_YAML --model yolo11n.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_n
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_DESP_YAML --model yolo11n.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_n_des
# small
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_YAML --model yolo11s.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_s
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_DESP_YAML --model yolo11s.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_s_des
# medium (batch 8)
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_YAML --model yolo11m.pt --epochs 80 --batch 8 --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_m
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BASE_DESP_YAML --model yolo11m.pt --epochs 80 --batch 8 --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_base_m_des


## Optimized (balanced vs denoised balanced) — n/s/m

In [ ]:
BAL_YAML='/content/fpus23_project/dataset/fpus23_yolo_balanced/data.yaml'
BAL_DESP_YAML='/content/fpus23_project/dataset/fpus23_yolo_balanced_despeckled/data.yaml'
# nano
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_YAML --model yolo11n.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_n
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_DESP_YAML --model yolo11n.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_n_des
# small
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_YAML --model yolo11s.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_s
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_DESP_YAML --model yolo11s.pt --epochs 80 --batch ${FPUS23_BATCH:-16} --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_s_des
# medium (batch 8)
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_YAML --model yolo11m.pt --epochs 80 --batch 8 --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_m
!python '/content/fpus23/scripts/train_yolo_fpus23_phase1.py' --data $BAL_DESP_YAML --model yolo11m.pt --epochs 80 --batch 8 --imgsz 768 --project $DRIVE_DIR --name fpus23_denoise_opt_m_des
